In [1]:
import os

import torch
from diffusers import StableDiffusionPipeline, DDPMScheduler, StableDiffusionInpaintPipeline
from osm_dataset import TextToImageDataset
from config import CHECKPOINTS_DIR, MODEL_NAME, DEVICE, DATA_DIR
from torch.utils.data import DataLoader


In [2]:

def get_prediction(pipe: StableDiffusionPipeline, prompt: str):
    with torch.no_grad():
        images = pipe(prompt, height=256, width=256).images

    return images[0]


In [ ]:
# !python diff_hugging.py
import diff_hugging
diff_hugging.main()

getting data


Resolving data files:   0%|          | 0/2145 [00:00<?, ?it/s]

Found cached dataset imagefolder (/home/marcin/.cache/huggingface/datasets/imagefolder/Wrocław, PL-16adbde1af182d57/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/marcin/.cache/huggingface/datasets/imagefolder/Wrocław, PL-16adbde1af182d57/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f/cache-794788cf658f2c34.arrow
The config attributes {'predict_epsilon': True} were passed to DDPMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


training...


/home/marcin/Documents/mgr/semestr_II/nlp/nlp_l2/nlp_venv/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
/home/marcin/Documents/mgr/semestr_II/nlp/nlp_l2/nlp_venv/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /home/marcin/Documents/mgr/projekt_n_w/own/models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                 | Params
------------------------------------------------------
0 | text_encoder | CLIPTextModel        | 123 M 
1 | vae          | AutoencoderKL        | 83.7 M
2 | unet         | UNet2DConditionModel | 579 M 
------------------------------------------------------
579 M     Trainable params
206 M     Non-trainable params
786

Training: 0it [00:00, ?it/s]

/home/marcin/Documents/mgr/semestr_II/nlp/nlp_l2/nlp_venv/lib/python3.10/site-packages/diffusers/configuration_utils.py:134: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDPMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDPMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)
Epoch 0, global step 2144: 'train/loss' reached 0.12045 (best 0.12045), saving model to '/home/marcin/Documents/mgr/projekt_n_w/own/models/stable_wroclaw_test-v1.ckpt' as top 1
Epoch 1, global step 4288: 'train/loss' reached 0.11832 (best 0.11832), saving model to '/home/marcin/Documents/mgr/projekt_n_w/own/models/stable_wroclaw_test-v1.ckpt' as top 1
Epoch 2, global step 6432: 'train/loss' was not in top 1
Epoch 3, global step 8576: 'train/loss' was not in top 1
Epoch 4, global step 10720: 'train/loss' was not in top 1
Epoch 

In [4]:
noise_scheduler = DDPMScheduler.from_pretrained(
        "models/wroclaw_v2", subfolder="scheduler"
    )
pipeline = StableDiffusionPipeline.from_pretrained("models/wroclaw_v2", safety_checker=None, scheduler=noise_scheduler).to(DEVICE)

/home/marcin/Documents/mgr/semestr_II/nlp/nlp_l2/nlp_venv/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
The config attributes {'dropout': 0.0} were passed to UNet2DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network 

In [5]:
ds = TextToImageDataset(DATA_DIR)
dl = DataLoader(ds, batch_size=1)

Resolving data files:   0%|          | 0/2145 [00:00<?, ?it/s]

Found cached dataset imagefolder (/home/marcin/.cache/huggingface/datasets/imagefolder/Wrocław, PL-16adbde1af182d57/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2144 [00:00<?, ? examples/s]

In [6]:
from tqdm import tqdm
import torch.nn.functional as F

def evaluate(dl, pipeline):
    total_loss = 0.0
    for img, caption in tqdm(dl):
        latents = pipeline.vae.encode(
                        img.to(DEVICE)
                    ).latent_dist.sample()
        latents = latents * pipeline.vae.config.scaling_factor

        # Sample noise that we'll add to the latents
        noise = torch.randn_like(latents)

        bsz = latents.shape[0]
        # Sample a random timestep for each image
        timesteps = torch.randint(
            0,
            noise_scheduler.config.num_train_timesteps,
            (bsz,),
            device=latents.device,
        )
        timesteps = timesteps.long()

        # Add noise to the latents according to the noise magnitude at each timestep
        # (this is the forward diffusion process)
        noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

        # Get the text embedding for conditioning
        encoder_hidden_states = pipeline.text_encoder(caption.to(DEVICE))[0]

        # Get the target for loss depending on the prediction type
        if noise_scheduler.config.prediction_type == "epsilon":
            target = noise
        elif noise_scheduler.config.prediction_type == "v_prediction":
            target = noise_scheduler.get_velocity(latents, noise, timesteps)
        else:
            raise ValueError(
                f"Unknown prediction type {noise_scheduler.config.prediction_type}"
            )

        # Predict the noise residual and compute loss
        model_pred = pipeline.unet(
            noisy_latents, timesteps, encoder_hidden_states
        ).sample

        
        loss = F.mse_loss(
            model_pred.float(), target.float(), reduction="mean"
            )
        total_loss += loss.item()
    return total_loss / len(dl)

In [7]:
evaluate(dl, pipeline)

100%|██████████| 2144/2144 [03:25<00:00, 10.45it/s]


0.12338643512209016